In [1]:
import websocket
import threading
import requests
import json
import datetime
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
asset_info_df = pd.read_csv('C:/Users/User/Desktop/dydx_cafg/live data/2023-03-28 09;20;56.csv')
asset_info_df.rename(columns={'Unnamed: 0': 'timestamp'}, inplace=True)
asset_info_df['timestamp'] = pd.to_datetime(asset_info_df['timestamp'], format='%Y-%m-%d %H;%M;%S')
asset_info_df

,timestamp,dydx_bid,dydx_ask,dydx_predicted_funding_rate,dydx_next_funding_rate_time,apollox_bid,apollox_ask,apollox_predicted_funding_rate,apollox_next_funding_rate_time,bybit_bid,bybit_ask,bybit_predicted_funding_rate
0,2023-03-28 09:20:56,26961.0,26962.0,8.800000e-08,1679968800000,26933.0,26934.0,-0.000009,1679990400000,26942.0,26942.1,-0.000018
1,2023-03-28 09:20:57,26961.0,26962.0,8.800000e-08,1679968800000,26933.0,26934.0,-0.000009,1679990400000,26942.0,26942.1,-0.000018
2,2023-03-28 09:20:59,26960.0,26961.0,8.800000e-08,1679968800000,26933.0,26934.0,-0.000009,1679990400000,26942.0,26942.1,-0.000018
3,2023-03-28 09:21:00,26955.0,26956.0,8.800000e-08,1679968800000,26930.0,26931.0,-0.000009,1679990400000,26937.3,26937.4,-0.000018
4,2023-03-28 09:21:01,26960.0,26961.0,8.800000e-08,1679968800000,26931.0,26932.0,-0.000009,1679990400000,26937.3,26937.4,-0.000018
...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2023-03-28 10:23:48,27032.0,27033.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045
3596,2023-03-28 10:23:49,27032.0,27033.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045
3597,2023-03-28 10:23:50,27031.0,27032.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045
3598,2023-03-28 10:23:51,27031.0,27032.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045


# a rough backtest
1. Use random signals for testing purposes, 1 = buy, -1 = sell, order amount not specified
2. Funding payment and transaction costs need further investigation

In [3]:
asset_info_df['dydx_signal'] = np.random.randint(-1, 2, asset_info_df.shape[0])
asset_info_df['apollox_signal'] = np.random.randint(-1, 2, asset_info_df.shape[0])
asset_info_df

,timestamp,dydx_bid,dydx_ask,dydx_predicted_funding_rate,dydx_next_funding_rate_time,apollox_bid,apollox_ask,apollox_predicted_funding_rate,apollox_next_funding_rate_time,bybit_bid,bybit_ask,bybit_predicted_funding_rate,dydx_signal,apollox_signal
0,2023-03-28 09:20:56,26961.0,26962.0,8.800000e-08,1679968800000,26933.0,26934.0,-0.000009,1679990400000,26942.0,26942.1,-0.000018,-1,1
1,2023-03-28 09:20:57,26961.0,26962.0,8.800000e-08,1679968800000,26933.0,26934.0,-0.000009,1679990400000,26942.0,26942.1,-0.000018,0,1
2,2023-03-28 09:20:59,26960.0,26961.0,8.800000e-08,1679968800000,26933.0,26934.0,-0.000009,1679990400000,26942.0,26942.1,-0.000018,1,-1
3,2023-03-28 09:21:00,26955.0,26956.0,8.800000e-08,1679968800000,26930.0,26931.0,-0.000009,1679990400000,26937.3,26937.4,-0.000018,0,1
4,2023-03-28 09:21:01,26960.0,26961.0,8.800000e-08,1679968800000,26931.0,26932.0,-0.000009,1679990400000,26937.3,26937.4,-0.000018,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,2023-03-28 10:23:48,27032.0,27033.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045,-1,0
3596,2023-03-28 10:23:49,27032.0,27033.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045,1,1
3597,2023-03-28 10:23:50,27031.0,27032.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045,-1,-1
3598,2023-03-28 10:23:51,27031.0,27032.0,-3.358000e-06,1679972400000,27008.0,27009.0,-0.000025,1679990400000,27004.8,27004.9,-0.000045,-1,-1


In [108]:
class Backtest:
    def __init__(self, asset_info, start_time, end_time, unit, buy_cost, sell_cost, extra_cost):
        self.asset_info = asset_info
        self.start_time = start_time
        self.end_time = end_time
        self.date_range = pd.date_range(start=start_time, end=end_time, freq='1s')

        self.unit = unit
        self.buy_cost = buy_cost
        self.sell_cost = sell_cost
        self.extra_cost = extra_cost

        self.cash = 0
        self.coin_dydx = 0
        self.coin_apollox = 0
        self.portfolio = pd.DataFrame(columns=['timestamp', 'cash', 'coin_dydx', 'coin_apollox', 'total_value'])

    def buy(self, time, exchange, value): # value divided by asset_info ask price, (assume taking the best ask price), exchange = dydx or apollox, value = self.unit
        coin_amount = value / float(self.asset_info[f'{exchange}_ask'][asset_info_df['timestamp'] == time])
        self.cash = self.cash - self.buy_cost*abs(value) - abs(value)
        if exchange == 'dydx':
            self.coin_dydx = self.coin_dydx + coin_amount
        elif exchange == 'apollox':
            self.coin_apollox = self.coin_apollox + coin_amount
        else:
            pass

    def sell(self, time, exchange, value): # value divided by asset_info bid price, exchange = dydx or apollox, value = self.unit
        coin_amount = value / float(self.asset_info[f'{exchange}_bid'][asset_info_df['timestamp'] == time])
        self.cash = self.cash - self.sell_cost*abs(value) + abs(value) 
        if exchange == 'dydx':
            self.coin_dydx = self.coin_dydx - abs(coin_amount)
        elif exchange == 'apollox':
            self.coin_apollox = self.coin_apollox - abs(coin_amount)
        else:
            pass

    #Trade execution function
    def execute_trade(self, row):
        for exchange in ['dydx', 'apollox']:
            if int(row[f'{exchange}_signal']) == 1:
                self.buy(time=row['timestamp'].values[0], exchange=exchange, value=self.unit)
            elif int(row[f'{exchange}_signal']) == -1:
                self.sell(time=row['timestamp'].values[0], exchange=exchange, value=self.unit)
            else:
                pass
    
    def funding_payment(self, time, exchange, position):
        for exchange in ['dydx', 'apollox']:
            # funding calculation (need further modification)
            payment = float(self.asset_info[f'{exchange}_funding'][asset_info_df['timestamp'] == time]) * position *self.asset_info[f'exchange_bid'][self.asset_info['timestamp']==time].values[0]
            self.cash = self.cash + payment

    #Main function
    def run_backtesting(self):
        for timepoint in self.date_range:
            order_book = self.asset_info[self.asset_info['timestamp']==timepoint]

            # basic trade execution
            try:
                self.execute_trade(row=order_book)
                Total_value = self.cash + self.coin_dydx*self.asset_info['dydx_bid'][self.asset_info['timestamp']==timepoint].values[0] + self.coin_apollox*self.asset_info['apollox_bid'][self.asset_info['timestamp']==timepoint].values[0]

                self.portfolio = self.portfolio.append({'timestamp': timepoint, 
                                                    'cash': self.cash, 
                                                    'coin_dydx': self.coin_dydx, 
                                                    'coin_apollox': self.coin_apollox, 
                                                    'total_value': Total_value}, ignore_index=True)
            except:
                print('Data not available for ' + str(timepoint))

            # if timepoint is 00:00:00, 08:00:00, 16:00:00, then call funding payment
            if timepoint.hour % 8 == 0:
                self.funding_payment(time=timepoint, exchange='dydx', position=self.coin_dydx)
                self.funding_payment(time=timepoint, exchange='apollox', position=self.coin_apollox)

In [109]:
start_time = pd.to_datetime('2023-03-28 09:30:00')
end_time = pd.to_datetime('2023-03-28 10:00:00')

bt = Backtest(asset_info=asset_info_df,
              start_time=start_time, 
              end_time=end_time,
              unit=100, 
              buy_cost=0.0004, 
              sell_cost=0.0004, 
              extra_cost=0)

In [110]:
bt.run_backtesting()

Data not available for 2023-03-28 09:30:01
Data not available for 2023-03-28 09:30:27
Data not available for 2023-03-28 09:30:56
Data not available for 2023-03-28 09:31:31
Data not available for 2023-03-28 09:32:03
Data not available for 2023-03-28 09:32:33
Data not available for 2023-03-28 09:32:59
Data not available for 2023-03-28 09:33:32
Data not available for 2023-03-28 09:34:02
Data not available for 2023-03-28 09:34:30
Data not available for 2023-03-28 09:34:54
Data not available for 2023-03-28 09:35:22
Data not available for 2023-03-28 09:35:48
Data not available for 2023-03-28 09:36:20
Data not available for 2023-03-28 09:36:44
Data not available for 2023-03-28 09:37:08
Data not available for 2023-03-28 09:37:26
Data not available for 2023-03-28 09:37:54
Data not available for 2023-03-28 09:38:16
Data not available for 2023-03-28 09:38:42
Data not available for 2023-03-28 09:39:06
Data not available for 2023-03-28 09:39:31
Data not available for 2023-03-28 09:39:57
Data not av

In [111]:
bt.portfolio.head(50)

,timestamp,cash,coin_dydx,coin_apollox,total_value
0,2023-03-28 09:30:00,99.96,0,-0.003705,-0.040000
1,2023-03-28 09:30:02,199.92,0,-0.007411,-0.080000
2,2023-03-28 09:30:03,299.88,-0.003702,-0.007411,-0.120000
3,2023-03-28 09:30:04,199.84,-0.003702,-0.003705,-0.163705
4,2023-03-28 09:30:05,199.76,-0.0,-0.007411,-0.247407
5,2023-03-28 09:30:06,399.68,-0.003702,-0.011116,-0.327407
6,2023-03-28 09:30:07,399.60,-0.007403,-0.007411,-0.411112
7,2023-03-28 09:30:08,199.52,-0.003702,-0.003706,-0.513324
8,2023-03-28 09:30:09,299.48,-0.007403,-0.003706,-0.553324
9,2023-03-28 09:30:10,299.48,-0.007403,-0.003706,-0.553324
